In [32]:
from transformers import *
import os
import copy
import numpy as np
import pandas as pd
import tokenizers

In [35]:
model_type = "roberta-base"

tokenizer =  RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )

# model_type = "albert-base-v2"

# tokenizer = AlbertTokenizer.from_pretrained(
#             pretrained_model_name_or_path="transformers_vocab/{}-spiece.model".format(model_type),
#             lowercase=True,
#         )

# model_type = "bert-base-cased"

# tokenizer =  BertTokenizer(
#             vocab_file="transformers_vocab/{}-vocab.txt".format(model_type),
#             lowercase=True,
#             add_prefix_space=True
#         )


def get_prediction_v3(tokenizer, tweet, selected_text):
    
    # lower cased here
    tweet = " ".join(str(tweet).split())
    selected_text = " ".join(str(selected_text).split())

    tweet = " " + tweet.lower()
    selected_text = " " + selected_text.lower()
    
    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None
    
    # get char idx
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind + len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break
    
    # get char mask
    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    # get word offsets
    tweet_offsets = []
    tweet_offsets_token_level = []
    cursor = 0
    input_ids_orig = []
    
    for word in tweet.split():
        
        sub_words = tokenizer.tokenize(" " + word)
        encoded_word = tokenizer.convert_tokens_to_ids(sub_words)
        orig_word = tokenizer.convert_tokens_to_string(sub_words)
#         print(sub_words, " " + word)
#         print(" " + word, orig_word)
        number_of_tokens = len(encoded_word)
        input_ids_orig += encoded_word
        
        start_offsets = cursor
        
        token_level_cursor = start_offsets
        
        for i in range(number_of_tokens):
            
            
            if (model_type == "bert-base-uncased") or (model_type == "bert-large-uncased") \
                or (model_type == "bert-base-cased") or (model_type == "bert-large-cased"):
                # for bert tokenizer, replace "##" and add " " for first sub_word
                sub_word_len = len(sub_words[i].replace("##", ""))
                if i == 0:
                    sub_word_len += 1
            else:
                sub_word_len = len(sub_words[i])

            if sub_words[i] == "Ã¯" or sub_words[i] == "Â½":
                sub_word_len = 1
            
            tweet_offsets_token_level.append((token_level_cursor, token_level_cursor + sub_word_len))
            cursor = token_level_cursor + sub_word_len
            token_level_cursor += sub_word_len
            
        end_offsets = cursor

        for i in range(number_of_tokens):

            tweet_offsets.append((start_offsets, end_offsets))
        
#     print(tweet_offsets)
#     print(tweet_offsets_token_level)
        
    
    # get word idx
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets_token_level):
#         print(tweet[offset1: offset2])
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]
    
#     print(targets_start, targets_end)
    print(tweet_offsets[targets_start][0], tweet_offsets_token_level[targets_start][0])
    print(tweet_offsets[targets_end][1], tweet_offsets_token_level[targets_end][1])
    
    
    return tweet[tweet_offsets[targets_start][0]: tweet_offsets[targets_end][1]], \
            tweet[tweet_offsets_token_level[targets_start][0]: tweet_offsets_token_level[targets_end][1]]

# tweet = "star wars ............ is **** boo??? i wanna do your job hand it over u can act as me at my high school lol"
# selected_text = "l"

tweet = "Happy Mother`s Day to all the Momma`s to be and Mommas currently.  Have a wonderful day!!!"
selected_text = "wonderful"

get_prediction_v3(tokenizer, tweet, selected_text)

73 73
83 83


(' wonderful', ' wonderful')

In [71]:
model_type = "roberta-base"

tokenizer_0 = RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
        )


tokenizer_1 = tokenizers.ByteLevelBPETokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )


tweet = "greetings"
tweet = " " + " ".join(tweet.split())
tweet_copy = copy.deepcopy(tweet)
tweet = tweet.lower()

input_ids_orig = []
tweet_offsets_word_level = []
tweet_offsets_token_level = []
cursor = 0


for i, word in enumerate(tweet.split()):

    sub_words = tokenizer_0.tokenize(" " + word)
    encoded_word = tokenizer_0.convert_tokens_to_ids(sub_words)
    number_of_tokens = len(encoded_word)
    input_ids_orig += encoded_word

    start_offsets = cursor

    token_level_cursor = start_offsets

    for i in range(number_of_tokens):

        if (model_type == "bert-base-uncased") or (model_type == "bert-large-uncased") \
            or (model_type == "bert-base-cased") or (model_type == "bert-large-cased"):
            # for bert tokenizer, replace "##" and add " " for first sub_word
            sub_word_len = len(sub_words[i].replace("##", ""))
            if i == 0:
                sub_word_len += 1
        else:
            sub_word_len = len(sub_words[i])
        
#         print(sub_words[i], len(sub_words[i]))
        if sub_words[i] == "Ã¯" or sub_words[i] == "Â½":
            sub_word_len = 1

        tweet_offsets_token_level.append((token_level_cursor, token_level_cursor + sub_word_len))
        cursor = token_level_cursor + sub_word_len
        token_level_cursor += sub_word_len

    end_offsets = cursor

    for i in range(number_of_tokens):

        tweet_offsets_word_level.append((start_offsets, end_offsets))

# print(input_ids_orig_0)
# print(tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(input_ids_orig_0)))
print(input_ids_orig)
print(tweet_offsets_word_level)
for i in range(len(tweet_offsets_word_level)):
    print(tweet[tweet_offsets_word_level[i][0]: tweet_offsets_word_level[i][1]], \
         tweet[tweet_offsets_token_level[i][0]: tweet_offsets_token_level[i][1]])
print(tweet_offsets_token_level)
print(tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(input_ids_orig)))

ids_1 = tokenizer_1.encode(tweet_copy).ids
print(ids_1)
print(tokenizer_1.encode(tweet_copy).offsets)
print(tokenizer_1.decode(ids_1))

[17395, 1033]
[(0, 10), (0, 10)]
 greetings  greet
 greetings ings
[(0, 6), (6, 10)]
 greetings
[17395, 1033]
[(0, 6), (6, 10)]
 greetings


In [4]:
train = pd.read_csv("/media/jionie/my_disk/Kaggle/Tweet/input/tweet-sentiment-extraction/train.csv")

In [5]:
# model_type = "albert-base-v2"

# tokenizer = AlbertTokenizer.from_pretrained(
#             pretrained_model_name_or_path="transformers_vocab/{}-spiece.model".format(model_type),
#             lowercase=True,
#         )

# model_type = "bert-base-cased"

# tokenizer =  BertTokenizer(
#             vocab_file="transformers_vocab/{}-vocab.txt".format(model_type),
#             lowercase=True,
#             add_prefix_space=True
#         )

model_type = "roberta-base"

tokenizer =  RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )

# for i in range(len(train)):
    
#     prediction, prediction_token = get_prediction_v3(tokenizer, train["text"].iloc[i], train["selected_text"].iloc[i])
#     prediction = prediction.strip()
#     prediction_token = prediction_token.strip()
#     print(prediction, "--------------------", prediction_token)
    
#     label = str(train["selected_text"].iloc[i]).lower().strip()
#     if label != prediction:
#         print(
#               label, \
#               "-------------------", \
#               get_prediction_v3(tokenizer, train["text"].iloc[i], train["selected_text"].iloc[i]), \
#               "-------------------", train["text"].iloc[i])

In [6]:
model_type = "roberta-base"

tokenizer_1 =  RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )

tokenizer_2 = tokenizers.ByteLevelBPETokenizer(
            vocab_file='/media/jionie/my_disk/Kaggle/Tweet/code/dataset/transformers_vocab/roberta-base-vocab.json',
            merges_file='/media/jionie/my_disk/Kaggle/Tweet/code/dataset/transformers_vocab/roberta-base-merges.txt',
            lowercase=True,
            add_prefix_space=True)


def check_offsets(tokenizer_1, tokenizer_2, tweet, selected_text):
    
    # lower cased here
    tweet = " ".join(str(tweet).split())
    selected_text = " ".join(str(selected_text).split())

    tweet = " " + tweet.lower()
    selected_text = " " + selected_text.lower()
    
    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None
    
    # get char idx
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind + len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break
    
    # get char mask
    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    # get word offsets
    tweet_offsets_token_level = []
    cursor = 0
    input_ids_orig = []
    
    for i, word in enumerate(tweet.split()):

        sub_words = tokenizer_1.tokenize(" " + word)
#         print(sub_words)
        encoded_word = tokenizer_1.convert_tokens_to_ids(sub_words)
        number_of_tokens = len(encoded_word)
        input_ids_orig += encoded_word

        start_offsets = cursor
        end_offsets = cursor

        token_level_cursor = start_offsets

        for i in range(number_of_tokens):

            if (model_type == "bert-base-uncased") or (model_type == "bert-large-uncased") \
                or (model_type == "bert-base-cased") or (model_type == "bert-large-cased"):
                # for bert tokenizer, replace "##" and add " " for first sub_word
                sub_word_len = len(sub_words[i].replace("##", ""))
                if i == 0:
                    sub_word_len += 1
            else:
                sub_word_len = len(sub_words[i])
            
            tweet_offsets_token_level.append((token_level_cursor, token_level_cursor + sub_word_len))
            cursor = token_level_cursor + sub_word_len
            token_level_cursor += sub_word_len

            
    tweet_offsets = tokenizer_2.encode(tweet).offsets
    
    if str(tweet_offsets_token_level) != str(tweet_offsets):
        print(tweet)
        print(tweet_offsets_token_level)
        print(tweet_offsets)
        
    
#     get word idx
#     target_idx = []
#     for j, (offset1, offset2) in enumerate(tweet_offsets_token_level):
# #         print(tweet[offset1: offset2])
#         if sum(char_targets[offset1: offset2]) > 0:
#             target_idx.append(j)
    
#     targets_start = target_idx[0]
#     targets_end = target_idx[-1]
    
# tweet = "iï¿½m trying to"
# selected_text = "iï¿½m trying to"
# check_offsets(tokenizer_1, tokenizer_2, tweet, selected_text)
# for i in range(len(train)):

#     check_offsets(tokenizer_1, tokenizer_2, train["text"].iloc[i], train["selected_text"].iloc[i])

In [8]:
model_type = "roberta-base"

tokenizer_1 =  RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )

tokenizer_2 = tokenizers.ByteLevelBPETokenizer(
            vocab_file='/media/jionie/my_disk/Kaggle/Tweet/code/dataset/transformers_vocab/roberta-base-vocab.json',
            merges_file='/media/jionie/my_disk/Kaggle/Tweet/code/dataset/transformers_vocab/roberta-base-merges.txt',
            lowercase=True,
            add_prefix_space=True)


def check_label(tokenizer_1, tokenizer_2, tweet, selected_text, num):
    
    # lower cased here
    tweet_with_extra_space = str(tweet).lower()
    tweet = "" + " ".join(str(tweet).split()).lower()
    selected_text = " " + " ".join(str(selected_text).split()).lower()
    original_tweet = copy.deepcopy(tweet)
    original_selected_text = copy.deepcopy(selected_text)
    
    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None
    
    # get char idx
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind + len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break
    
    # get char mask
    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    # get word offsets
    tweet_offsets_token_level = []
    cursor = 0
    input_ids_orig = []
    
    for i, word in enumerate(tweet.split()):

        sub_words = tokenizer_1.tokenize(" " + word)
        encoded_word = tokenizer_1.convert_tokens_to_ids(sub_words)
        number_of_tokens = len(encoded_word)
        input_ids_orig += encoded_word

        start_offsets = cursor
        end_offsets = cursor

        token_level_cursor = start_offsets

        for i in range(number_of_tokens):

            if (model_type == "bert-base-uncased") or (model_type == "bert-large-uncased") \
                or (model_type == "bert-base-cased") or (model_type == "bert-large-cased"):
                # for bert tokenizer, replace "##" and add " " for first sub_word
                sub_word_len = len(sub_words[i].replace("##", ""))
                if i == 0:
                    sub_word_len += 1
            else:
                sub_word_len = len(sub_words[i])
            
            tweet_offsets_token_level.append((token_level_cursor, token_level_cursor + sub_word_len))
            cursor = token_level_cursor + sub_word_len
            token_level_cursor += sub_word_len

    tweet_offsets_token_level = tokenizer_2.encode(tweet).offsets
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets_token_level):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]
    
#     pred = tweet[tweet_offsets_token_level[targets_start][0]: tweet_offsets_token_level[targets_end][1]]
    
#     pred = str(pred).lower().strip()
#     selected_text = str(selected_text).lower().strip()
    
#     if pred != selected_text:
#         print(pred)
#         print(selected_text)
#         num += 1

    prediction = tweet[tweet_offsets_token_level[targets_start][0]: tweet_offsets_token_level[targets_end][1]].strip()
    filtered_output = ""
    finished = False

    # print("tweet: ", tweet)
    # print("prediction: ", prediction)

    if len(prediction) == 0:
        filtered_output = prediction
    else:
        max_match = 0
        for idx in range(len(original_tweet)):
            curr_output = ""
            if original_tweet[idx] == prediction[0]:
                tweet_idx = idx
                for prediction_idx in range(len(prediction)):
                    if (original_tweet[tweet_idx] == prediction[prediction_idx]):
                        curr_output += original_tweet[tweet_idx]
                        tweet_idx += 1

                        # reach tweet end
                        if (tweet_idx >= len(original_tweet)):
                            if (prediction_idx == len(prediction) - 1):
                                finished = True
                            break

                        # reach prediction end
                        if (prediction_idx == len(prediction) - 1):
                            finished = True
                            break
                    else:
                        # skip extra " "
                        if (prediction[prediction_idx] == " "):
                            continue
                        else:
                            break
                if finished:
                    filtered_output = curr_output
                    break
                else:
                    if max_match < len(curr_output):
                        max_match = len(curr_output)
                        filtered_output = curr_output
                        
    if filtered_output.strip() != original_selected_text.strip():
        print("-----------")
        print("-----------")
        print("-----------")
        print(prediction)
        print(selected_text)
        print(filtered_output)
        print(original_selected_text)
        print("-----------")
        print("-----------")
        print("-----------")
        num += 1
    
    return num

num = 0
for i in range(len(train)):

    num = check_label(tokenizer_1, tokenizer_2, train["text"].iloc[i], train["selected_text"].iloc[i], num)
    
print(num)

-----------
-----------
-----------
bullying me.
 bullying me
bullying me.
 bullying me
-----------
-----------
-----------
-----------
-----------
-----------
gonna
 onna
gonna
 onna
-----------
-----------
-----------
-----------
-----------
-----------
..no internet
 .no internet
..no internet
 .no internet
-----------
-----------
-----------
-----------
-----------
-----------
well so much for being unhappy for about 10 minutes.
 well so much for being unhappy for about 10 minute
well so much for being unhappy for about 10 minutes.
 well so much for being unhappy for about 10 minute
-----------
-----------
-----------
-----------
-----------
-----------
a little happy for
 a little happy fo
a little happy for
 a little happy fo
-----------
-----------
-----------
-----------
-----------
-----------
dont like going
 dont like go
dont like going
 dont like go
-----------
-----------
-----------
-----------
-----------
-----------
and i`m not thrilled at all with mine.
 d i`m not thri

-----------
-----------
-----------
...i`m sorry
 .i`m sorry
...i`m sorry
 .i`m sorry
-----------
-----------
-----------
-----------
-----------
-----------
...no one will help me
 .no one will help me
...no one will help me
 .no one will help me
-----------
-----------
-----------
-----------
-----------
-----------
me! gonna scare rachel with quarantine tonight, this shall be fun
 ! gonna scare rachel with quarantine tonight, this shall be fun
me! gonna scare rachel with quarantine tonight, this shall be fun
 ! gonna scare rachel with quarantine tonight, this shall be fun
-----------
-----------
-----------
-----------
-----------
-----------
ok thank
 k thank
ok thank
 k thank
-----------
-----------
-----------
-----------
-----------
-----------
witter. he gives twitter tips. hope this helps
 . he gives twitter tips. hope this helps
witter. he gives twitter tips. hope this helps
 . he gives twitter tips. hope this helps
-----------
-----------
-----------
-----------
-----------


-----------
-----------
-----------
..she`s such an amazing singer
 she`s such an amazing singer
..she`s such an amazing singer
 she`s such an amazing singer
-----------
-----------
-----------
-----------
-----------
-----------
for the best job in the world. good luck.
 r the best job in the world. good luck.
for the best job in the world. good luck.
 r the best job in the world. good luck.
-----------
-----------
-----------
-----------
-----------
-----------
hey i loved acs but i had to see it online, is not here yet cant wait to buy it!!! you look amazing there
 y i loved acs but i had to see it online, is not here yet cant wait to buy it!!! you look amazing ther
hey i loved acs but i had to see it online, is not here yet cant wait to buy it!!! you look amazing there
 y i loved acs but i had to see it online, is not here yet cant wait to buy it!!! you look amazing ther
-----------
-----------
-----------
-----------
-----------
-----------
understand, pains been part of my life f

-----------
-----------
-----------
together fairly well!
 er fairly well!
together fairly well!
 er fairly well!
-----------
-----------
-----------
-----------
-----------
-----------
'they say i look yummy
 they say i look yummy
'they say i look yummy
 they say i look yummy
-----------
-----------
-----------
-----------
-----------
-----------
charger.sorry
 .sorry
charger.sorry
 .sorry
-----------
-----------
-----------
-----------
-----------
-----------
and its not that cold ahahah
 d its not that cold ahahah
and its not that cold ahahah
 d its not that cold ahahah
-----------
-----------
-----------
-----------
-----------
-----------
2010, i think or very late 2009! its just wrong
 , i think or very late 2009! its just wrong
2010, i think or very late 2009! its just wrong
 , i think or very late 2009! its just wrong
-----------
-----------
-----------
-----------
-----------
-----------
.can`t believe they`re divorcing
 can`t believe they`re divorcing
.can`t believe they`re d

that sucks.
 t sucks
-----------
-----------
-----------
-----------
-----------
-----------
sad that
 sad th
sad that
 sad th
-----------
-----------
-----------
-----------
-----------
-----------
and missed everythin
 d missed everythin
and missed everythin
 d missed everythin
-----------
-----------
-----------
-----------
-----------
-----------
well that does make me feel better about staying up even later to watch the daily show. lay on the guilt again.
 well that does make me feel better about staying up even later to watch the daily show. lay on the guilt agai
well that does make me feel better about staying up even later to watch the daily show. lay on the guilt again.
 well that does make me feel better about staying up even later to watch the daily show. lay on the guilt agai
-----------
-----------
-----------
-----------
-----------
-----------
could motivate
 uld motiva
could motivate
 uld motiva
-----------
-----------
-----------
-----------
-----------
-----------
i`m

...

-----------
-----------
-----------
sa. probably spelt it wrong lol.
 . probably spelt it wrong lol.
sa. probably spelt it wrong lol.
 . probably spelt it wrong lol.
-----------
-----------
-----------
-----------
-----------
-----------
i hate
 i ha
i hate
 i ha
-----------
-----------
-----------
-----------
-----------
-----------
omg garbo fake playing during one of the songs!
 omg garbo fake playing during one of the songs
omg garbo fake playing during one of the songs!
 omg garbo fake playing during one of the songs
-----------
-----------
-----------
-----------
-----------
-----------
byeeeee!!!
 byeeeee!!
byeeeee!!!
 byeeeee!!
-----------
-----------
-----------
-----------
-----------
-----------
looks cool.
 s cool
looks cool.
 s cool
-----------
-----------
-----------
-----------
-----------
-----------
do u hav a garden? bunny probs r decidedly uncool. thanks 4d link!
 do u hav a garden? bunny probs r decidedly uncool. thanks 4d link
do u hav a garden? bunny probs r decid

-----------
-----------
-----------
you deserve so much better.
 ou deserve so much bette
you deserve so much better.
 ou deserve so much bette
-----------
-----------
-----------
-----------
-----------
-----------
really well
 y well
really well
 y well
-----------
-----------
-----------
-----------
-----------
-----------
having golden times
 g golden times
having golden times
 g golden times
-----------
-----------
-----------
-----------
-----------
-----------
hurt so badly??!
 hurt so badly??
hurt so badly??!
 hurt so badly??
-----------
-----------
-----------
-----------
-----------
-----------
nice outside...w
 nice outside..
nice outside...w
 nice outside..
-----------
-----------
-----------
-----------
-----------
-----------
much fun.
 h fun
much fun.
 h fun
-----------
-----------
-----------
-----------
-----------
-----------
my pleasure...
 my pleasure..
my pleasure...
 my pleasure..
-----------
-----------
-----------
-----------
-----------
-----------
just ignore


-----------
-----------
-----------
soooooooo what happened to power forreal?
 soooooooo what happened to power forrea
soooooooo what happened to power forreal?
 soooooooo what happened to power forrea
-----------
-----------
-----------
-----------
-----------
-----------
..best
 best
..best
 best
-----------
-----------
-----------
-----------
-----------
-----------
girls love
 ls lo
girls love
 ls lo
-----------
-----------
-----------
-----------
-----------
-----------
do stupid
 o stupid
do stupid
 o stupid
-----------
-----------
-----------
-----------
-----------
-----------
**** comes home & is going to make a mess of it...m
 **** comes home & is going to make a mess of it..
**** comes home & is going to make a mess of it...m
 **** comes home & is going to make a mess of it..
-----------
-----------
-----------
-----------
-----------
-----------
away sadfaced.
 y sadfaced
away sadfaced.
 y sadfaced
-----------
-----------
-----------
-----------
-----------
-----------
tast

-----------
-----------
-----------
back. sad
 . sad
back. sad
 . sad
-----------
-----------
-----------
-----------
-----------
-----------
well, i was under the impression most people have the urge to buy new sofas to be fair, we`re not tiling, we have a man for that
 well, i was under the impression most people have the urge to buy new sofas to be fair, we`re not tiling, we have a man for th
well, i was under the impression most people have the urge to buy new sofas to be fair, we`re not tiling, we have a man for that
 well, i was under the impression most people have the urge to buy new sofas to be fair, we`re not tiling, we have a man for th
-----------
-----------
-----------
-----------
-----------
-----------
.. don`t blame me
 . don`t blame me
.. don`t blame me
 . don`t blame me
-----------
-----------
-----------
-----------
-----------
-----------
m down!
 m down
m down!
 m down
-----------
-----------
-----------
-----------
-----------
-----------
food, its hard to tell
 

-----------
-----------
-----------
yea but that`s an old pic, she looks a lot different now...she turned into such a beautiful women i miss her a lot!
 yea but that`s an old pic, she looks a lot different now...she turned into such a beautiful women i miss her a lo
yea but that`s an old pic, she looks a lot different now...she turned into such a beautiful women i miss her a lot!
 yea but that`s an old pic, she looks a lot different now...she turned into such a beautiful women i miss her a lo
-----------
-----------
-----------
-----------
-----------
-----------
ic, sorry
 c, sorr
ic, sorry
 c, sorr
-----------
-----------
-----------
-----------
-----------
-----------
..happy
 happy
..happy
 happy
-----------
-----------
-----------
-----------
-----------
-----------
much appreciate
 ch appreciat
much appreciate
 ch appreciat
-----------
-----------
-----------
-----------
-----------
-----------
ja, enjoyyitverymuch
 ja, enjoyyitverymu
ja, enjoyyitverymuch
 ja, enjoyyitverymu
----

In [ ]:
tokenizer_2 = tokenizers.ByteLevelBPETokenizer(
            vocab_file='/media/jionie/my_disk/Kaggle/Tweet/code/dataset/transformers_vocab/roberta-base-vocab.json',
            merges_file='/media/jionie/my_disk/Kaggle/Tweet/code/dataset/transformers_vocab/roberta-base-merges.txt',
            lowercase=True,
            add_prefix_space=True)

def get_target_idx(tweet, selected_text, offsets):
    
    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1

    target_idx = []
    for j, (offset1, offset2) in enumerate(offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)

    start_idx = target_idx[0]
    end_idx = target_idx[-1]

    return start_idx, end_idx


def check_label_v2(tokenizer_2, tweet, selected_text):
    
    # lower cased here
    tweet = " ".join(str(tweet).split())
    selected_text = " ".join(str(selected_text).split())

    tweet = " " + tweet.lower()
    selected_text = " " + selected_text.lower()
    
    tweet_offsets_token_level = tokenizer_2.encode(tweet).offsets

    targets_start, targets_end = get_target_idx(tweet, selected_text,tweet_offsets_token_level)
    
    pred = tweet[tweet_offsets_token_level[targets_start][0]: tweet_offsets_token_level[targets_end][1]+1]
    
    pred = str(pred).lower().strip()
    selected_text = str(selected_text).lower().strip()
    
    if pred != selected_text:
        print(pred)
        print(selected_text)

for i in range(len(train)):

    check_label_v2(tokenizer_2, train["text"].iloc[i], train["selected_text"].iloc[i])

In [ ]:
tokenizer_1.add_tokens(["ï", "¿", "½"])

tweet = "Well thatï¿½s disappointing to hear."
selected_text = "s disappointing"

# lower cased here
tweet = " ".join(str(tweet).split()).lower()
selected_text = " ".join(str(selected_text).split()).lower()
original_tweet = copy.deepcopy(tweet)
original_selected_text = copy.deepcopy(selected_text)

# some token optimization
special_tokens = ["!!!!", "!!!", "!!", "!", \
                  "....", "...", "..", ".", \
                  "????", "???", "??", "?", \
                  "****", "***", "**", "*", \
                  "----", "---", "--", "-", \
                  "~~~~", "~~~", "~~", "~", \
                  "////", "///", "//", "/", \
                  "````", "```", "``", "`", \
                  "====", "===", "==", "=", \
                  "####", "###", "##", "#", \
                  "&&&&", "&&&", "&&", "&", \
                  "))))", ")))", "))", ")", \
                  ";;;;", ";;;", ";;", ";", \
                  "''''", "'''", "''", "'", \
                  "<<<<", "<<<", "<<", "<"
                 ]
for token in special_tokens:
    replace_token = " ".join(token) + " "
    tweet = tweet.replace(token, replace_token)
    selected_text = selected_text.replace(token, replace_token)

tweet = " " + " ".join(tweet.split())
selected_text = " " + " ".join(selected_text.split())

len_st = len(selected_text) - 1
idx0 = None
idx1 = None

# get char idx
for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
    if " " + tweet[ind: ind + len_st] == selected_text:
        idx0 = ind
        idx1 = ind + len_st - 1
        break

# get char mask
char_targets = [0] * len(tweet)
if idx0 != None and idx1 != None:
    for ct in range(idx0, idx1 + 1):
        char_targets[ct] = 1
print(tweet[idx0: idx1 + 1])
# get word offsets
tweet_offsets_token_level = []
tweet_offsets_word_level = []
cursor = 0
input_ids_orig = []

print(tweet)
for i, word in enumerate(tweet.split()):
    
    sub_words = tokenizer_1.tokenize(" " + word)
    print(sub_words)
    encoded_word = tokenizer_1.convert_tokens_to_ids(sub_words)
    number_of_tokens = len(encoded_word)
    input_ids_orig += encoded_word

    start_offsets = cursor
    end_offsets = cursor

    token_level_cursor = start_offsets

    for i in range(number_of_tokens):
        
        tweet_offsets_word_level.append((start_offsets, end_offsets))

        if (model_type == "bert-base-uncased") or (model_type == "bert-large-uncased") \
            or (model_type == "bert-base-cased") or (model_type == "bert-large-cased"):
            # for bert tokenizer, replace "##" and add " " for first sub_word
            sub_word_len = len(sub_words[i].replace("##", ""))
            if i == 0:
                sub_word_len += 1
        else:
            sub_word_len = len(sub_words[i])
        
        if sub_words[i] == "Ã¯":
            sub_word_len = 1
        print(sub_words[i], len(sub_words[i]), tweet[token_level_cursor: token_level_cursor + sub_word_len])
        tweet_offsets_token_level.append((token_level_cursor, token_level_cursor + sub_word_len))
        cursor = token_level_cursor + sub_word_len
        token_level_cursor += sub_word_len

# tweet_offsets_token_level = tokenizer_2.encode(tweet).offsets
target_idx = []
for j, (offset1, offset2) in enumerate(tweet_offsets_token_level):
    if sum(char_targets[offset1: offset2]) > 0:
        target_idx.append(j)

targets_start = target_idx[0]
targets_end = target_idx[-1]


prediction = tweet[tweet_offsets_token_level[targets_start][0]: tweet_offsets_token_level[targets_end][1]].strip()
print(prediction)

filtered_output = ""
finished = False

# print("tweet: ", tweet)
# print("prediction: ", prediction)

if len(prediction) == 0:
    filtered_output = prediction
else:
    max_match = 0
    for idx in range(len(original_tweet)):
        curr_output = ""
        if original_tweet[idx] == prediction[0]:
            tweet_idx = idx
            for prediction_idx in range(len(prediction)):
                if (original_tweet[tweet_idx] == prediction[prediction_idx]):
                    curr_output += original_tweet[tweet_idx]
                    tweet_idx += 1

                    # reach tweet end
                    if (tweet_idx >= len(original_tweet)):
                        if (prediction_idx == len(prediction) - 1):
                            finished = True
                        break

                    # reach prediction end
                    if (prediction_idx == len(prediction) - 1):
                        finished = True
                        break
                else:
                    # skip extra " "
                    if (prediction[prediction_idx] == " "):
                        continue
                    else:
                        break
            if finished:
                filtered_output = curr_output
                break
            else:
                if max_match < len(curr_output):
                    max_match = len(curr_output)
                    filtered_output = curr_output

print(filtered_output, original_selected_text)

In [ ]:
model_type = "roberta-base"

tokenizer_1 =  RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )

tokenizer_2 = tokenizers.ByteLevelBPETokenizer(
            vocab_file='/media/jionie/my_disk/Kaggle/Tweet/code/dataset/transformers_vocab/roberta-base-vocab.json',
            merges_file='/media/jionie/my_disk/Kaggle/Tweet/code/dataset/transformers_vocab/roberta-base-merges.txt',
            lowercase=True,
            add_prefix_space=True)


def check_white_space(tokenizer_1, tokenizer_2, tweet, selected_text, num):
    
    # lower cased here
    tweet_with_extra_space = str(tweet).lower()
    tweet = "" + " ".join(str(tweet).split()).lower()
    selected_text = " " + " ".join(str(selected_text).split()).lower()
    original_tweet = copy.deepcopy(tweet)
    original_selected_text = copy.deepcopy(selected_text)
    
    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None
    
    # get char idx
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind + len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break
    
    # get char mask
    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    # get word offsets
    tweet_offsets_token_level = []
    cursor = 0
    input_ids_orig = []
    
    for i, word in enumerate(tweet.split()):

        sub_words = tokenizer_1.tokenize(" " + word)
        encoded_word = tokenizer_1.convert_tokens_to_ids(sub_words)
        number_of_tokens = len(encoded_word)
        input_ids_orig += encoded_word

        start_offsets = cursor
        end_offsets = cursor

        token_level_cursor = start_offsets

        for i in range(number_of_tokens):

            if (model_type == "bert-base-uncased") or (model_type == "bert-large-uncased") \
                or (model_type == "bert-base-cased") or (model_type == "bert-large-cased"):
                # for bert tokenizer, replace "##" and add " " for first sub_word
                sub_word_len = len(sub_words[i].replace("##", ""))
                if i == 0:
                    sub_word_len += 1
            else:
                sub_word_len = len(sub_words[i])
            
            tweet_offsets_token_level.append((token_level_cursor, token_level_cursor + sub_word_len))
            cursor = token_level_cursor + sub_word_len
            token_level_cursor += sub_word_len

    tweet_offsets_token_level = tokenizer_2.encode(tweet).offsets
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets_token_level):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]
    
#     pred = tweet[tweet_offsets_token_level[targets_start][0]: tweet_offsets_token_level[targets_end][1]]
    
#     pred = str(pred).lower().strip()
#     selected_text = str(selected_text).lower().strip()
    
#     if pred != selected_text:
#         print(pred)
#         print(selected_text)
#         num += 1

    prediction = tweet[tweet_offsets_token_level[targets_start][0]: tweet_offsets_token_level[targets_end][1]].strip()
    filtered_output = ""
    finished = False

    # print("tweet: ", tweet)
    # print("prediction: ", prediction)

    if len(prediction) == 0:
        filtered_output = prediction
    else:
        max_match = 0
        for idx in range(len(original_tweet)):
            curr_output = ""
            if original_tweet[idx] == prediction[0]:
                tweet_idx = idx
                for prediction_idx in range(len(prediction)):
                    if (original_tweet[tweet_idx] == prediction[prediction_idx]):
                        curr_output += original_tweet[tweet_idx]
                        tweet_idx += 1

                        # reach tweet end
                        if (tweet_idx >= len(original_tweet)):
                            if (prediction_idx == len(prediction) - 1):
                                finished = True
                            break

                        # reach prediction end
                        if (prediction_idx == len(prediction) - 1):
                            finished = True
                            break
                    else:
                        # skip extra " "
                        if (prediction[prediction_idx] == " "):
                            continue
                        else:
                            break
                if finished:
                    filtered_output = curr_output
                    break
                else:
                    if max_match < len(curr_output):
                        max_match = len(curr_output)
                        filtered_output = curr_output
                        
    if filtered_output.strip() != original_selected_text.strip():
        print("-----------")
        print("-----------")
        print("-----------")
        print(prediction)
        print(selected_text)
        print(filtered_output)
        print(original_selected_text)
        print("-----------")
        print("-----------")
        print("-----------")
        num += 1
    
    return num

num = 0
for i in range(len(train)):

    num = check_label(tokenizer_1, tokenizer_2, train["text"].iloc[i], train["selected_text"].iloc[i], num)
    
print(num)

In [14]:
def get_extra_space(text):
    
    text = str(text)
    num_space = 0
    
    for char in text:
        if char == " ":
            num_space += 1
            
    num_words = len(text.split())
    extra_space = num_space - num_words + 1
    
    return extra_space

train["extra_sapce"] = train["text"].apply(lambda x: get_extra_space(x))

In [15]:
train["extra_sapce"].value_counts()

 0     11087
 1     10933
 2      4131
 3       899
 4       239
 5       114
 6        37
 7        13
 8        10
 9         5
 15        2
 10        2
 14        2
 11        1
 20        1
 12        1
 19        1
-3         1
 13        1
 95        1
Name: extra_sapce, dtype: int64

In [22]:
train[train["extra_sapce"]==7]["text"].values

array(['   Wow what a beautiful picture... and by the way....I am straight....just wanted to let you know!!!     Bella',
       'This is my favourite shirt, because its true.  <3 grumpy.    a) cute  b) huggable  c) life of the party  (d)... http://tinyurl.com/q7eax7',
       ' What??  Nooooooooo      There goes my weekend!',
       '    _Geronimo    Thanks for sharing with your friends!',
       'Im bumbed though cuz ima miss da laker game        |double sad face|',
       ' Oh noooooo!!!      I`m so sorry!!  That sucks hard!',
       'is  happy  cause  my  voice  is  back  <3',
       'to jodies and played sims till my daddy picked me up        and that is what i did yesterday. basically it was my social time',
       ' no    I kind of miss my first main.   Male Tauren Shaman named Icewater stuck at 61.  Maybe I`ll fetch him one day.',
       'Be beautiful        http://www.theskinsociety.com',
       ' am alive   but at work   finish at 1300   sun is out',
       'awake sadly        

In [23]:
train[train["extra_sapce"]==7]["selected_text"].values

array(['Wow what a beautiful pictur', 'favourite',
       'What??  Nooooooooo      There goes my week', 'imo',
       'Im bumbed though cuz ima miss da laker game        |double sad',
       'That', 'happy',
       'to jodies and played sims till my daddy picked me up        and that is what i did yesterday. basically it was my social',
       'of mis', 'Be beautiful',
       'am alive   but at work   finish at 1300   sun', 'sadly',
       'poor Stuart...    I was wonde'], dtype=object)

In [52]:
tweet = '   Wow what a beautiful picture... and by the way....I am straight....just wanted to let you know!!!     Bella'
selected_text = 'Wow what a beautiful pictur'
tweet_with_extra_space = str(tweet).lower()
tweet = " " + " ".join(str(tweet).split()).lower()
selected_text = " " + " ".join(str(selected_text).split()).lower()

print(tweet, selected_text)

len_st = len(selected_text) - 1
idx0 = None
idx1 = None

# get char idx
for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
    if " " + tweet[ind: ind + len_st] == selected_text:
        idx0 = ind
        idx1 = ind + len_st - 1
        break
        
# get char mask
char_targets = [0] * len(tweet)
if idx0 != None and idx1 != None:
    for ct in range(idx0, idx1 + 1):
        char_targets[ct] = 1
        
tweet_offsets_word_level = []
cursor = 0

for i, word in enumerate(tweet.split()):
    
    start_offsets = cursor
    cursor += len(" " + word)
    end_offsets = cursor

    tweet_offsets_word_level.append((start_offsets, end_offsets))

        
target_idx = []
for j, (offset1, offset2) in enumerate(tweet_offsets_word_level):
    if sum(char_targets[offset1: offset2]) > 0:
        target_idx.append(j)

targets_start = target_idx[0]
targets_end = target_idx[-1]
tweet_with_extra_space = " " + tweet_with_extra_space
selected_words = tweet_with_extra_space[tweet_offsets_word_level[targets_start][0]:tweet_offsets_word_level[targets_end][1]]
print(selected_words.strip())
# start_idx = tweet_with_extra_space.find(selected_words.strip())

# end_idx = 0
# for i in range(word_idx, len(tweet_with_extra_space)):
#     if tweet_with_extra_space[word_idx: i].strip() == selected_words.strip():
#         end_idx = i
#         break
# tweet = tweet.strip()
# print(tweet[start_idx: end_idx])

 wow what a beautiful picture... and by the way....i am straight....just wanted to let you know!!! bella  wow what a beautiful pictur
wow what a beautiful picture
